In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import pandas as pd
from tensorflow.keras.preprocessing.text import  text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import re
import pickle

In [7]:
df = pd.read_csv('/content/translated.csv')
df.head()

,Unnamed: 0,en_comment,fa_comment,label
0,0,Really sorry for the time to write your servic...,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,1,"It was supposed to arrive for 2 hours, but it ...",قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,2,The price of this model is not compatible with...,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,3,"Great, all right and good quality, I hope you ...",عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,4,Vanilla sweetness was just a model.,شیرینی وانیلی فقط یک مدل بود.,0.0


In [13]:
df['label'] = ~df['label'].astype(bool)
df.head()

,Unnamed: 0,en_comment,fa_comment,label
0,0,Really sorry for the time to write your servic...,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,False
1,1,"It was supposed to arrive for 2 hours, but it ...",قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,True
2,2,The price of this model is not compatible with...,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,False
3,3,"Great, all right and good quality, I hope you ...",عالللی بود همه چه درست و به اندازه و کیفیت خوب...,True
4,4,Vanilla sweetness was just a model.,شیرینی وانیلی فقط یک مدل بود.,True


In [18]:
df['label'] = df['label'].astype(int)
df.head()

,Unnamed: 0,en_comment,fa_comment,label
0,0,Really sorry for the time to write your servic...,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,0
1,1,"It was supposed to arrive for 2 hours, but it ...",قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,1
2,2,The price of this model is not compatible with...,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,0
3,3,"Great, all right and good quality, I hope you ...",عالللی بود همه چه درست و به اندازه و کیفیت خوب...,1
4,4,Vanilla sweetness was just a model.,شیرینی وانیلی فقط یک مدل بود.,1


In [21]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
X = normalize_texts(df['en_comment'])

In [22]:
y = df['label']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.1, random_state=42)

In [28]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/tokenizer.pickle' , 'rb') as f:
  tokenizer = pickle.load(f)

In [29]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/vectorizer.pickle' , 'rb') as f:
  vectorizer = pickle.load(f)

In [30]:
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [31]:
MAX_LENGTH = 255 

In [32]:
X_train_seq = pad_sequences(X_train_seq, maxlen=MAX_LENGTH)
X_test_seq = pad_sequences(X_test_seq, maxlen=MAX_LENGTH)

In [35]:
model = load_model('/content/drive/MyDrive/persian-sentiment-analysis/fianl-model-dualchanal.h5')

In [38]:
model.fit([X_train_seq ,X_train_tfidf.toarray() ],y_train, epochs=10 , validation_data=([X_test_seq , X_test_tfidf.toarray()] , y_test))

Epoch 1/10
1948/1948 [==============================] - 56s 28ms/step - loss: 0.3825 - accuracy: 0.8305 - val_loss: 0.3682 - val_accuracy: 0.8363
Epoch 2/10
1948/1948 [==============================] - 55s 28ms/step - loss: 0.3343 - accuracy: 0.8558 - val_loss: 0.3646 - val_accuracy: 0.8386
Epoch 3/10
1948/1948 [==============================] - 57s 29ms/step - loss: 0.3018 - accuracy: 0.8722 - val_loss: 0.3813 - val_accuracy: 0.8393
Epoch 4/10
1948/1948 [==============================] - 55s 28ms/step - loss: 0.2681 - accuracy: 0.8888 - val_loss: 0.3802 - val_accuracy: 0.8390
Epoch 5/10
1948/1948 [==============================] - 54s 28ms/step - loss: 0.2338 - accuracy: 0.9039 - val_loss: 0.4300 - val_accuracy: 0.8387
Epoch 6/10
1948/1948 [==============================] - 54s 28ms/step - loss: 0.2033 - accuracy: 0.9154 - val_loss: 0.5111 - val_accuracy: 0.8316
Epoch 7/10
1948/1948 [==============================] - 56s 29ms/step - loss: 0.1773 - accuracy: 0.9263 - val_loss: 0.6523 -